In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# Data Set and Motivation

Since the mid-1980s the amount spent on political elections by campaigns and outside groups, measured by the Federal Election Committee, has grown by 555 percent nationwide. While voting ultimately determines the result of any given election, analyzing political contributions can signal how passionate and invested individuals are in the success of a given candidate. In this report we will assess the political climate in Colorado through focusing on individual campaign finance contributions from 1992 through 2016.  

The data used in this report is from the Center for Responsive Politics (CRP), an independent and nonprofit research group that collects political finance data. All 50 states require disclosures to be filed as a means of transparency and as a means to regulate money spent in elections. These disclosures are filed through the Federal Election Committee (FEC), the independent regulatory agency charged with administering and enforcing federal campaign finance law. The CRP uses public FEC fillings as the source for their data. The metadata used in this report can be found in the link below:
https://www.opensecrets.org/resources/datadictionary/UserGuide.pdf

Our report will utilize two data tables accross seven political cycles. The first table is individual contribution data, which includes 23 attributes including contribution amounts and political affiliation of recipient. Additionally, we will utilize the candidate table which includes 12 attributes and provides all candidites who recieved contributions in a given cycle. Samples of the two tables will be shown in subsequent pages. 

# Actual Task and Research Questions

The political landscape of Colorado has evolved rapidly over the last quarter century. While once a solidly conservative state, Colorado’s changing culture and influx of new residents from other regions has shifted Colorado’s politics more progressive. As shown in the chart below, both houses of Colorado's state assembly were typically controlled by Republicans from 1992 through the mid 2000's. However, since the late 2000's through 2013, Democrats have controlled both houses in addition to the Governer's seat. 



<img src="screen_shots/col_political_trends chart.png">
*Source: https://ballotpedia.org/Colorado_General_Assembly*

We set out to use campaign finance data to observe how these trends over the last 25 years have affected how Coloradans contribute to political campaigns by considering the following research questions:
* What trends exist in magnitude of money given, number of donations and political affiliation?
* Is there a relationship between the amount of money donated and candidate success in elections?
* How have political contributions evolved in different regions of the state?

# Data Cleaning
    - used %%script false to not run certain code blocks

We were able to download each year’s datasets from the website; however, the datafiles were too big to upload into GitHub. The first step in our data cleaning process was to convert the datafiles into a more compact form to allow us to upload all the data into our GitHub repository. We decided to convert the data text files into pickle files. Since each campaign year contained the same four datafiles, we created four functions in our CleanUP.py code, to read in the corresponding text files into a pandas data frame to clean up and then convert the cleaned data frame into a pickle file. When reading in each text file to convert to a data frame, we had to generate our own delimiter since the text files were not separated by commas.

In [ ]:
# Example of the uncleaned individuals 1992 file
! head indivs92.txt

The individuals text file was the largest of the datafiles. Since we only wanted to focus on the state of Colorado, once the text file was read in and converted into a data frame, we only pulled out the individuals that matched with ‘CO’ in the State column. This made it faster to clean the data frame because not all the data separated into their respective columns. The amount and date columns were still grouped with the RealCode column values. Once the original data frame was created, we had to then separate those columns. Then we were able to put the data frame into a pickle file.

In [ ]:
# Example of the dataframe of the cleaned individuals 1992 file
df = pd.read_pickle('analysis_of_crp_data/data/CampaignFin92/indivs92.pkl')
df

From the pacs text file not all the data separated into their respective columns either. The amount, date, and RealCode columns were still grouped with the CID column values. Once the original data frame was created, we had to then separate those columns. Then we were able to put the data frame into a pickle file.

The candidates text file generated a data frame that was separated correctly once the file was read in. Two symbols had to be removed from each of the end columns. Then we were able to put the data frame into a pickle file.

The pac to pac text file had many inconsistent missing values, this made it difficult to find the correct separator that would not raise a value error of length of rows not equal with one another. Since the pac to pac data files contained many unfilled values, we decided not to use this data and focus on the larger individuals data set along with the candidates and pacs.

We then were able to upload the three pickle files from each year to our GitHub repository. We took those pickle files to read in and convert into data frames. We compiled those data frames to create a complete individuals data frame and a candidates data frame of all the years.

The individuals data frame consisted of only Colorado. We still had to pull out the candidates from Colorado. The data frame had an Orgname column with values where the first two characters were the state and the last two characters represent the office sought for. We pulled out the candidates that contained ‘CO’ in that column.

In [ ]:
%%script false
cand_df.DistIDRunFor.str.contains('CO')
candCO_df = cand_df.loc[cand_df.DistIDRunFor.str.contains('CO')]
candCO_df.index = np.arange(len(candCO_df))

In [ ]:
%%script false
# Convert column to string remove duplicate information, 'CO' and party in Win/Loss column
candCO_df.loc[:,'DistIDRunFor'] = candCO_df.loc[:,'DistIDRunFor'].astype(str)
candCO_df.loc[:,'DistIDRunFor'] = candCO_df.loc[:,'DistIDRunFor'].apply(lambda x: pd.Series(str(x).replace('CO','')))
candCO_df.loc[:,'WinLoss'] = candCO_df.loc[:,'WinLoss'].apply(lambda x: pd.Series(str(x).replace(x[:1],'')))

In order to run visualizations on the individuals of Colorado data frame, we had to change the Amount column datatype from an object to an integer so we could perform statistic summarys on it.

In [ ]:
%%script false
# change the dtype of Amount from object to integer
indivs_df['Amount'] = indivs_df['Amount'].apply(lambda x: int(x))

The RecipCode column in the individuals data frame contained the party the individual contributed to and if that candidate won or loss. To pull out this information we first had to assign None to missing values in the RecipCode column.

In [ ]:
%%script false
import re

for i in range(len(indivs_df['RecipCode'])):
    att1 = re.search(r'[A-Z]|[0-9]',str(indivs_df.RecipCode[i]))
    if att1 == None:
        indivs_df.RecipCode[i] = None

Once there were no missing values, we were able to split the RecipCode into two columns since the value was two characters, Party and Result. We pulled out the first character which represented the party, it was either a capital letter or a number and the second character was assigned to the result column. If there was no second character, then NaN was assigned.

In [ ]:
%%script false
# split the RecipCode 
sub_df = indivs_df['RecipCode'].apply(lambda x: re.findall('[A-Z]|[0-9]',str(x)))

# create a new df with Party and Result column and add values to them
df= pd.DataFrame(columns=['Party','Result'])
df['Party'] = sub_df.apply(lambda x: x[0])

result = []
for i in sub_df:
    if len(i) < 2:
        result.append(np.nan)
    else:
        result.append(i[1])
df['Result'] = result

indivs_df['RecipCode']=df['Party']
indivs_df.insert(14,'Results',df['Result'])
indivs_df = indivs_df.rename(columns={'RecipCode':'Party'})